In [1]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
import os

#import the Cross sections file 
openmc.config['cross_sections'] = "/home/f_z/endfb-vii.1-hdf5/cross_sections.xml"

In [ ]:
#Usefull functions for the simulation 


def cleaner(directory="."):
    """
    Removes all files in the specified directory whose names 
    contain 'statepoint' or 'summary'.

    Parameters:
    - directory (str): The folder where files should be deleted (default: current directory).
    """
    files_deleted = 0
    for filename in os.listdir(directory):  # Loop through files in directory
        if "statepoint" in filename or "summary" in filename:
            file_path = os.path.join(directory, filename)  # Full file path
            try:
                os.remove(file_path)  # Delete file
                files_deleted += 1
                print(f" Deleted: {filename}")
            except Exception as e:
                print(f" Error deleting {filename}: {e}")
    if files_deleted == 0:
         print("No matching files found in the directory.")

def check_enrichment(enrichment, enrichment_pos):
    '''
    It's a function to control the avg value of the enrichment of the assembly.
    -Enrichment is an np.array, it gives the values of enrichment used 
    -enrichment_pos gives a np.array, it reports the number of assembly per enrichment
    '''
    avg_enrichment=0.
    total_pins=264
    for i in range (len(enrichment)):
        avg_enrichment=avg_enrichment+enrichment[i]*enrichment_pos[i]/264
    return avg_enrichment
    
    


In [ ]:
#ZIRCALOY
'''
enr=[3.5,4.5,6]
###Uranium###
uo2=openmc.Material(material_id=1)
uo2.add_element('U',1,enrichment=enr[1]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2.add_element('O',2)
uo2.set_density('g/cm3',10.3)

uo2_lower_enr=openmc.Material(material_id=2)
uo2_lower_enr.add_element('U',1,enrichment=enr[0]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2_lower_enr.add_element('O',2)
uo2_lower_enr.set_density('g/cm3',10.3)

uo2_higher_enr=openmc.Material(material_id=3)
uo2_higher_enr.add_element('U',1,enrichment=enr[2]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2_higher_enr.add_element('O',2)
uo2_higher_enr.set_density('g/cm3',10.3)

###Coolant###

water=openmc.Material(material_id=4)
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3',0.650)

###Assembly###

helium=openmc.Material(material_id=5)
helium.add_element('He',1)
helium.set_density('g/cm3',0.0043)


###Cladding### (from Input Correlations for Irradiation Creep of FeCrAl and SiC Based...)
#and Analysis of Options andExperimental Examination of Fuels for Water Cooled Reactors with Increased Accident Tolerance (ACTOF)

zircaloy=openmc.Material(material_id=6)
zircaloy.add_element('Zr',0.9843,'wo')
zircaloy.add_element('Sn',0.0120,'wo')
zircaloy.add_element('Fe',0.0240,'wo')
zircaloy.add_element('Cr',0.0120,'wo')
zircaloy.set_density('g/cm3',6.57)
#print(zircaloy)

#plugs
steel= openmc.Material(material_id=7)
steel.add_element('C',0.0005,'wo')
steel.add_element('Ti',0.005,'wo')
steel.add_element('Cr',0.1625,'wo')
steel.add_element('Mn',0.02,'wo')
steel.add_element('Fe',0.65,'wo')
steel.add_element('Ni',0.14,'wo')
steel.add_element('Mo',0.022,'wo')
steel.set_density('g/cm3',7.55)

materials=openmc.Materials([uo2,uo2_lower_enr,uo2_higher_enr,helium,water,zircaloy,steel])
materials.export_to_xml()

###Geometry of the single pellet###

#lateral surface, infinite cylinder centered in z axis, outer surfaces
frod_height=385.866  
r_p=0.857/2
r_g=r_p+0.016
r_c=r_g+0.0570
print(2*r_c)
s_pellet=openmc.ZCylinder(0,0,r_p)
s_gap=openmc.ZCylinder(0,0,r_g)
s_clad=openmc.ZCylinder(0,0,r_c)

#top and bottom surface
#-rodheight/2(b_bottom_plug)#bottom_plug_pos#pellet_stack#bottom_plenum_pos#top_plenum_pos(t_bottom_plug)#rodheight/2(t_top_plug)#

s_top=openmc.ZPlane(frod_height/2., boundary_type='vacuum') #reflective measn that we don't have leakages
s_bottom=openmc.ZPlane(-frod_height/2, boundary_type='vacuum') 

###full rod geometry, plenum only on the upper part of the rod
bottom_plug_pos=-frod_height/2+1.465
bottom_plenum_pos=frod_height/2-16.457-1.468
top_plenum_pos=frod_height/2-1.468

s_bottom_plenum= openmc.ZPlane(bottom_plenum_pos) 
s_top_plenum= openmc.ZPlane(top_plenum_pos)
s_b_top_end_plug= openmc.ZPlane(bottom_plug_pos) #surface between bottom plug and fuel stack

#let's create a geometry to run the code:
rod_pitch=1.26
#edge = rod_pitch/2 * 1/np.sin(np.pi/3)
#hex_cell = openmc.model.SPrism(edge_length=edge,orientation='x')#,boundary_type='reflective')
square_cell = openmc.model.RectangularPrism(rod_pitch,rod_pitch)#boundary_type='reflective')

###Cell construction ###
c_pellet=openmc.Cell(name="Pellet")
c_pellet.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet 
c_pellet.fill=uo2

c_gap=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap.fill=helium

# gas plenum
c_gas= openmc.Cell(name='gas plenum')
c_gas.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas.fill= helium

c_clad=openmc.Cell(name="Clad")
c_clad.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad.fill=zircaloy

### Coolant cell ###
c_coolant=openmc.Cell(name="Coolant")
c_coolant.region= +s_bottom & -s_top & +s_clad & -square_cell
c_coolant.fill=water

#bottom plug
c_b_plug= openmc.Cell(name='Bottom plug')
c_b_plug.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug.fill= steel

#top plug
c_t_plug= openmc.Cell(name='top plug')
c_t_plug.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug.fill= steel

fuel_rod_universe=openmc.Universe(cells=[c_pellet,c_gap,c_gas,c_clad,c_coolant,c_t_plug,c_b_plug])
'''

In [ ]:
#enrichemnt calculation

In [ ]:
#FECRAL

enr=[4.75,5.21,4.98]

###Uranium###
uo2=openmc.Material(material_id=1)
uo2.add_element('U',1,enrichment=enr[1]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2.add_element('O',2)
uo2.set_density('g/cm3',10.3)

uo2_lower_enr=openmc.Material(material_id=2)
uo2_lower_enr.add_element('U',1,enrichment=enr[0]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2_lower_enr.add_element('O',2)
uo2_lower_enr.set_density('g/cm3',10.3)

uo2_higher_enr=openmc.Material(material_id=3)
uo2_higher_enr.add_element('U',1,enrichment=enr[2]) #automatically it knows that the enrichment is 5%,it uses the natural abbundance 
uo2_higher_enr.add_element('O',2)
uo2_higher_enr.set_density('g/cm3',10.3)


###Coolant###
water=openmc.Material(material_id=4)
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3',0.650)

###Assembly###

helium=openmc.Material(material_id=5)
helium.add_element('He',1)
helium.set_density('g/cm3',0.0043)


###Cladding### (from Input Correlations for Irradiation Creep of FeCrAl and SiC Based...)
#and Analysis of Options andExperimental Examination of Fuels for Water Cooled Reactors with Increased Accident Tolerance (ACTOF)

fecral=openmc.Material(material_id=6)
fecral.add_element('Fe',0.7868,'wo')
fecral.add_element('Cr',0.1302,'wo')
fecral.add_element('Al',0.0508,'wo')
fecral.add_element('Mo',0.0199,'wo')
fecral.add_element('Si',0.0021,'wo')
fecral.add_element('Nb',0.0097,'wo')
fecral.add_element('Y',0.00032,'wo')
fecral.add_element('C',0.00003,'wo')
fecral.add_element('N',0.000013,'wo')
fecral.add_element('O',0.000028,'wo')
fecral.add_element('S',0.000003,'wo')
fecral.set_density('g/cm3',7.10)
#print(fecral)

#plugs
steel= openmc.Material(material_id=7)
steel.add_element('C',0.0005,'wo')
steel.add_element('Ti',0.005,'wo')
steel.add_element('Cr',0.1625,'wo')
steel.add_element('Mn',0.02,'wo')
steel.add_element('Fe',0.65,'wo')
steel.add_element('Ni',0.14,'wo')
steel.add_element('Mo',0.022,'wo')
steel.set_density('g/cm3',7.55)

materials=openmc.Materials([uo2,uo2_lower_enr,uo2_higher_enr,helium,water,fecral,steel])
materials.export_to_xml()

material_colors = {
    uo2: "orange", 
    uo2_higher_enr: "red",
    uo2_lower_enr:"yellow",
    steel: "darkgray",         
    fecral: "silver",         
    helium: "green", 
    water: "purple",        
}
###Geometry###

frod_height=385.866  
rod_pitch=1.26
r_p=0.857/2
r_g=r_p+0.016
r_c=r_g+0.0570
print(2*r_c)

s_pellet=openmc.ZCylinder(0,0,r_p)
s_gap=openmc.ZCylinder(0,0,r_g)
s_clad=openmc.ZCylinder(0,0,r_c)

#top and bottom surface
#-rodheight/2(b_bottom_plug)#bottom_plug_pos#pellet_stack#bottom_plenum_pos#top_plenum_pos(t_bottom_plug)#rodheight/2(t_top_plug)#

s_top=openmc.ZPlane(frod_height/2., boundary_type='vacuum')
s_bottom=openmc.ZPlane(-frod_height/2, boundary_type='vacuum') 

###full rod geometry, plenum only on the upper part of the rod
bottom_plug_pos=-frod_height/2+1.465
bottom_plenum_pos=frod_height/2-16.457-1.468
top_plenum_pos=frod_height/2-1.468

s_bottom_plenum= openmc.ZPlane(bottom_plenum_pos) 
s_top_plenum= openmc.ZPlane(top_plenum_pos)
s_b_top_end_plug= openmc.ZPlane(bottom_plug_pos) #surface between bottom plug and fuel stack

#coolant surroundings the fuel pin
square_cell = openmc.model.RectangularPrism(rod_pitch,rod_pitch)#boundary_type='reflective')
###Cell construction ###

#pellet
c_pellet=openmc.Cell(name="Pellet")
c_pellet.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet 
c_pellet.fill=uo2

c_gap=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap.fill=helium

# gas plenum
c_gas= openmc.Cell(name='gas plenum')
c_gas.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas.fill= helium

c_clad=openmc.Cell(name="Clad")
c_clad.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad.fill=fecral

### Coolant cell ###
c_coolant=openmc.Cell(name="Coolant")
c_coolant.region= +s_bottom & -s_top & +s_clad & -square_cell
c_coolant.fill=water

#bottom plug
c_b_plug= openmc.Cell(name='Bottom plug')
c_b_plug.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug.fill= steel

#top plug
c_t_plug= openmc.Cell(name='top plug')
c_t_plug.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug.fill= steel

fuel_rod_universe=openmc.Universe(cells=[c_pellet,c_gap,c_gas,c_clad,c_coolant,c_t_plug,c_b_plug])


In [ ]:
#Higher enrichment regions
#cell with hiegher enrichment
c_pellet_high_enr=openmc.Cell(name="Pellet")
c_pellet_high_enr.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet
c_pellet_high_enr.fill=uo2_higher_enr

c_gap_high_enr=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap_high_enr.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap_high_enr.fill=helium

# gas plenum
c_gas_high_enr=openmc.Cell(name='gas plenum')
c_gas_high_enr.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas_high_enr.fill= helium

c_clad_high_enr=openmc.Cell(name="Clad")
c_clad_high_enr.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad_high_enr.fill=fecral

### Coolant cell ###
c_coolant_high_enr=openmc.Cell(name="Coolant")
c_coolant_high_enr.region= +s_bottom & -s_top & +s_clad & -square_cell
c_coolant_high_enr.fill=water

#bottom plug
c_b_plug_high_enr=openmc.Cell(name='Bottom plug')
c_b_plug_high_enr.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug_high_enr.fill= steel

#top plug
c_t_plug_high_enr=openmc.Cell(name='top plug')
c_t_plug_high_enr.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug_high_enr.fill= steel

fuel_rod_high_enr_universe=openmc.Universe(cells=[c_pellet_high_enr,c_gap_high_enr,c_gas_high_enr,
                                                  c_clad_high_enr,c_coolant_high_enr,c_t_plug_high_enr,c_b_plug_high_enr]) #univere rod that will have higher enrichment

#high_enr_pos= np.empty((0, 2), dtype=int) 
high_enr_pos = []
low_enr_pos= []
# Define the corner positions (one quadrant)
corner_positions = [
    (0, 0), (0, 1), (0, 2), (0, 3),
    (1, 0), 
    (2, 0), 
    (3, 0),
    
]

internal_ring_pos = [
             
             (1, 1), (1, 2), (1, 3),
             (2, 1), 
             (3, 1),
]

# Apply symmetry to all four quadrants
for i, j in corner_positions:
    # Store each mirrored position
    high_enr_pos.append((i, j))               # Top-left
    high_enr_pos.append((i, 16 - j))          # Top-right
    high_enr_pos.append((16 - i, j))          # Bottom-left
    high_enr_pos.append((16 - i, 16 - j))     # Bottom-right

# Convert list to numpy array
high_enr_pos = np.array(high_enr_pos)
for i in range (17): #filling the position with higher enrichment
    for j in range(17):
        if (i==0 or i==16 or j==0 or j==16): #selects only the external boundary 
            new_pos=np.array([[i,j]])
            high_enr_pos=np.vstack([high_enr_pos, new_pos])
#print("high enr pos",high_enr_pos)

c_pellet_low_enr=openmc.Cell(name="Pellet")
c_pellet_low_enr.region=+s_b_top_end_plug & -s_bottom_plenum & -s_pellet
c_pellet_low_enr.fill=uo2_higher_enr

c_gap_low_enr=openmc.Cell(name="Gap") #gas in the gap along the fuel stack
c_gap_low_enr.region=+s_pellet & +s_b_top_end_plug & -s_bottom_plenum & -s_gap 
c_gap_low_enr.fill=helium

# gas plenum
c_gas_low_enr=openmc.Cell(name='gas plenum')
c_gas_low_enr.region=  +s_bottom_plenum & - s_top_plenum & -s_gap
c_gas_low_enr.fill= helium

c_clad_low_enr=openmc.Cell(name="Clad")
c_clad_low_enr.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad_low_enr.fill=fecral

### Coolant cell ###
c_coolant_low_enr=openmc.Cell(name="Coolant")
c_coolant_low_enr.region= +s_bottom & -s_top & +s_clad & -square_cell
c_coolant_low_enr.fill=water

#bottom plug
c_b_plug_low_enr=openmc.Cell(name='Bottom plug')
c_b_plug_low_enr.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_b_plug_low_enr.fill= steel

#top plug
c_t_plug_low_enr=openmc.Cell(name='top plug')
c_t_plug_low_enr.region=  +s_top_plenum  & -s_top & -s_gap
c_t_plug_low_enr.fill= steel

fuel_rod_low_enr_universe=openmc.Universe(cells=[c_pellet_low_enr,c_gap_low_enr,c_gas_low_enr,
                                                  c_clad_low_enr,c_coolant_low_enr,c_t_plug_low_enr,c_b_plug_low_enr]) #univere rod that will have higher enrichment

guide_tubes_pos = np.array([
                  [2, 5],   [2, 8],   [2, 11],
             [3, 3],                        [3, 13],
        [5, 2],   [5, 5],   [5, 8],   [5, 11],    [5, 14],
        [8, 2],   [8, 5],   [8, 8],   [8, 11],    [8, 14],
        [11, 2],  [11, 5],  [11, 8],  [11, 11],   [11, 14],
             [13, 3],                       [13, 13],
                  [14, 5],  [14, 8],  [14, 11]
    ])



for i, j in internal_ring_pos:
    # Store each mirrored position
    low_enr_pos.append((i, j))               # Top-left
    low_enr_pos.append((i, 16 - j))          # Top-right
    low_enr_pos.append((16 - i, j))          # Bottom-left
    low_enr_pos.append((16 - i, 16 - j))     # Bottom-right

low_enr_pos = np.array(low_enr_pos)  # Empty array with two columns for x, y positions

for i in range(17): 
    for j in range(17):  
        '''if (i + j) % 2 == 0 and [i, j] not in guide_tubes_pos.tolist():  # Chessboard pattern condition excluding the gt
            new_pos = np.array([[i, j]])  
            low_enr_pos = np.vstack([low_enr_pos, new_pos]) '''
        if (i==1 or i==15 or j==1 or j==15) and not (i==0 or i==16 or j==0 or j==16): #selects only the external boundary 
            new_pos=np.array([[i,j]])
            low_enr_pos=np.vstack([low_enr_pos, new_pos])

# Print the final list of positions
#print(len(low_enr_pos))

#check for the mean enrichment:


enr_rods=[len(low_enr_pos),264-len(high_enr_pos)-len(low_enr_pos),len(high_enr_pos)]
print("AVG enrichment",check_enrichment(enr,enr_rods))
print(enr_rods)

In [ ]:
#guide tubes definition 

c_guide_tubes_inner = openmc.Cell(name='innerGT')
c_guide_tubes_inner.region=-s_gap & -s_bottom_plenum & +s_b_top_end_plug
c_guide_tubes_inner.fill=water
# thimbles
c_thimbles= openmc.Cell(name='thimbles')
c_thimbles.region=  +s_bottom_plenum & - s_top & -s_gap
c_thimbles.fill= steel

c_coolant_gt=openmc.Cell(name="Coolant_gt")
c_coolant_gt.region= +s_bottom & -s_top & +s_clad & -square_cell# define a hexagonal region for the coolant
c_coolant_gt.fill=water

c_clad_gt=openmc.Cell(name="Clad")
c_clad_gt.region=+s_gap & +s_bottom & -s_top& -s_clad
c_clad_gt.fill=fecral

c_plug_gt= openmc.Cell(name='Bottom plug')
c_plug_gt.region=  -s_b_top_end_plug & + s_bottom & -s_gap
c_plug_gt.fill= steel

guide_tubes = openmc.Universe(name='Guide Tube')
guide_tubes.add_cells([c_guide_tubes_inner, c_plug_gt, c_thimbles,c_clad_gt,c_coolant_gt])

In [ ]:
#let's define the parameters for the construction of the assembly (squared 17*17)
 
bundle_pitch=17*rod_pitch

outer_boundary = openmc.model.RectangularPrism(
        bundle_pitch, bundle_pitch, boundary_type='reflective')

#let's create the bundle
bundle_lattice= openmc.RectLattice(name='Fuel Assembly')
bundle_lattice.pitch = (rod_pitch, rod_pitch)
bundle_lattice.lower_left = (-bundle_pitch/2, -bundle_pitch/2)


bundle_lattice.universes = np.full((17, 17), fuel_rod_universe)
bundle_lattice.universes[guide_tubes_pos[:, 0], guide_tubes_pos[:, 1]] =guide_tubes
#print("ENR--->",high_enr_pos[:, 0], high_enr_pos[:, 1])
#print("GT--->",guide_tubes_pos[:,0],guide_tubes_pos[:,1])

#if you want to include only the flat enrichment every where comment the following 2 lines
bundle_lattice.universes[high_enr_pos[:, 0], high_enr_pos[:, 1]] =fuel_rod_high_enr_universe
bundle_lattice.universes[low_enr_pos[:, 0], low_enr_pos[:, 1]] =fuel_rod_low_enr_universe

bundle_cell = openmc.Cell(fill=bundle_lattice, region= -outer_boundary & -s_top & +s_bottom)

bundle_universe= openmc.Universe (cells= [bundle_cell])
#bundle_universe.plot(basis='xz',width=(22,390), pixels=(700,700), colors=  material_colors,origin =[0,0,0])
bundle_universe.plot(basis='xy',width=(24,24), pixels=(1800,1800), color_by="material",colors=material_colors,origin =[0,0,0])

### import the geometry in a xml file

fuel_cell_geom=openmc.Geometry(bundle_universe)
fuel_cell_geom.export_to_xml()


In [ ]:
#Tallies
#let's define the "filters"
#in the parentesis we have the energy groups in Ev (0-->0.68; 0.68-->20*10^6)
#energies = np.logspace(np.log10(1e-5), np.log10(20.0e6), 201)
#e_filter=openmc.EnergyFilter(energies, filter_id=1)

e_filter=openmc.EnergyFilter([.0,.68,20e6])
msh_filter=openmc.RegularMesh(mesh_id=1)
msh_filter.dimension=(17,17,1)
#we define the most negative and positive point creating a parallelepide that comprehends everything
#we define the most negative and the most positive corner
msh_filter.lower_left= [-bundle_pitch/2, -bundle_pitch/2,-frod_height/2]
msh_filter.upper_right= [bundle_pitch/2, bundle_pitch/2,frod_height/2]
mesh_filter= openmc.MeshFilter(msh_filter,filter_id=2) 


flux= openmc.Tally(tally_id=1,name= 'Flux') # units are particle-cm per source particle.
flux.scores= ['flux']
flux.filters= [e_filter, mesh_filter]
'''
#to plot the spectrum of the flux

flux_energy= openmc.Tally(tally_id=1,name= 'Flux_en')
flux_energy.scores= ['flux']
flux_energy.filters= [e_filter]
'''

# Define tally for fission power
peaking = openmc.Tally(tally_id=2,name='fissionpower')
peaking.scores = ['kappa-fission']  # The recoverable energy production rate due to fission. 
peaking.filters = [mesh_filter]  # Apply mesh filter

fission_rr =openmc.Tally (tally_id=3,name='Fission reaction rate')
fission_rr.scores = ['fission']
fission_rr.filters = [e_filter, mesh_filter]

tallies = openmc.Tallies([flux,peaking,fission_rr])
tallies.export_to_xml()
#!cat.tallies.xml

In [ ]:
#simulation settings

source_point= openmc.stats.Point(xyz=(0.0,0.0,0.0))

#this represents only where the starting gen of neutrons will appear
source_region= openmc.stats.Box((-bundle_pitch/2,-bundle_pitch/2,-1.),(bundle_pitch/2,bundle_pitch/2,1.))
source_energy= openmc.stats.Watt( a=988000.0,b=2.249e-06)
point_source = openmc.Source(space=source_point,energy=source_energy)
source = openmc.Source(space=source_region,energy=source_energy)



#batches determination 

#we are performing a criticality calculation, for this reason we impose 50000 neutrons for each batch
sim_settings= openmc.Settings()
sim_settings.run_mode='eigenvalue'
sim_settings.particles =100000
sim_settings.inactive=20
sim_settings.batches=100
sim_settings.source= source

#we export also the simulation settings in te xml file
sim_settings.export_to_xml()
#!cat settings.xml

for cell in fuel_cell_geom.get_all_cells().values():
    print(f"Cell ID: {cell.id}, Region: {cell.region}")
#the output provide a dictionary with the list of different cell that are used to create the 
#universe (in this case fuel_rod_universe). The numbers inside parenthesis rappresents the surfaces
#that composed each cell, with negative and positive def based on what you did in the definition 
#of each region



In [ ]:
#run the simulation

'''
import time
    - name (str): A label for the case to differentiate runs.
    - N (int): Number of test cases (batch values).
    - batch_range (tuple): (min_batches, max_batches) to test.
    - inactive (int), the percentage of inactive batches
    - source, the one that i defined in the simulation settings cell.
    - particles (int): Number of particles per batch.
    
name="test"
N=10
batch_range=(30,50)
particles=50000
inactive=40

start_time = time.time()  # Record start time

optimize_batches(name, N, batch_range, inactive, point_source, particles)

end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Compute elapsed time
print(f"Elapsed time: {elapsed_time:.4f} seconds")
'''
cleaner()
openmc.run()

In [ ]:
#evaluate the ppf
sp_file = openmc.StatePoint("statepoint.100.h5")

ppf_tally = sp_file.get_tally(id=2)
fission_power = ppf_tally.get_values()

# Compute power peaking factor
max_power = fission_power.max()
#avg_power = fission_power.mean()
avg_power = fission_power.sum()/264 #to avoid the counting of the guide tubes

ppf = max_power / avg_power
print(fission_power.shape)
print(f'Power Peaking Factor: {ppf:.3f}')
print(f'MAx_power: {max_power:.3f}')


In [ ]:
#flux vs location
flux_tally=sp_file.get_tally(id=1)
flux_values = flux_tally.get_values().flatten()  # collapses the multidim array inro one dimension
#print("Flux values tally dimension:",flux_values.shape) # as you can see the tally is redefined as a one dim array
thermal_flux_values = flux_values[:289]
#print(thermal_flux_values)
# export the mesh
mesh_filter =flux_tally.find_filter(openmc.MeshFilter)
mesh = mesh_filter.mesh 
print("Mesh tally:",type(mesh)) #is a  class: 'openmc.mesh.RegularMesh' 

# the coordinates for the mesh
x_vals = np.linspace(mesh.lower_left[0], mesh.upper_right[0], mesh.dimension[0])
y_vals = np.linspace(mesh.lower_left[1], mesh.upper_right[1], mesh.dimension[1])
print("Len of the coordinates array along x/y",len(x_vals)) #x and y are arrays defined as (starting value, final value, number of subintervals)
print("Values inside the arrays",x_vals) # the coordinates are thus defined as (-bundle_pitch/2,bundle_pitch/2, 17)

# Reshape flux data to match mesh grid, passes from 289->17x17
flux_grid = thermal_flux_values.reshape(mesh.dimension[0], mesh.dimension[1])
print("Flux reshaped in the grid",flux_grid.shape)

# Choose a middle row to analyze
mid_row = (mesh.dimension[1] // 2)+1  # fix a y coordinate 

plt.figure(figsize=(8, 5))
plt.bar(x_vals, flux_grid[mid_row, :], width=0.5, color='blue', alpha=0.7, label="Flux")
#for that fixed coordinate it's possible to plot all the possible values along x ([mid_row,:])
print("Flux values along mid_row:",flux_grid[mid_row, :])
plt.xlabel("x [cm]")
plt.ylabel("Thermal flux[(eV*cm)^-1]")
plt.title("Flux Profile along x fixed y")
plt.legend()
#plt.ylim(0.,0.02)
plt.grid(True)

# Choose a middle column
mid_col = (mesh.dimension[0] // 2)+1  # Middle column

plt.figure(figsize=(8, 5))
plt.bar(y_vals, flux_grid[:, mid_col], width=0.5, color='red', alpha=0.7, label="Flux")
plt.xlabel("Y Coordinate [cm]")
plt.ylabel("Flux ")
plt.title("Flux Profile along y fixed x")
plt.legend()
plt.ylim(0.,0.02)
plt.grid(True)

#Plotting the peaking factors and power distribution maps
#creating the list of power peak per each coordinates
#power vs location, already imported in fission_power 
fission_power.flatten()  # collapses the multidim array inro one dimension
#print(fission_power)
# export the mesh
mesh_filter =ppf_tally.find_filter(openmc.MeshFilter)
mesh = mesh_filter.mesh 
print("Mesh tally:",type(mesh)) #is a  class: 'openmc.mesh.RegularMesh' 

# the coordinates for the mesh
x_vals = np.linspace(mesh.lower_left[0], mesh.upper_right[0], mesh.dimension[0])
y_vals = np.linspace(mesh.lower_left[1], mesh.upper_right[1], mesh.dimension[1])
print("Len of the coordinates array along x/y",len(x_vals)) #x and y are arrays defined as (starting value, final value, number of subintervals)
print("Values inside the arrays",x_vals) # the coordinates are thus defined as (-bundle_pitch/2,bundle_pitch/2, 17)

# Reshape flux data to match mesh grid, passes from 289->17x17
power_grid = fission_power.reshape(mesh.dimension[0], mesh.dimension[1])
print("Flux reshaped in the grid",power_grid.shape)


ppf_grid = power_grid / avg_power 
ppf_list = []
for i in range(mesh.dimension[0]):
    for j in range(mesh.dimension[1]):
        
        if not any((guide_tubes_pos == [i, j]).all(axis=1)):
            ppf_value = ppf_grid[i, j]
            x_coord = x_vals[i]
            y_coord = y_vals[j]
            ppf_list.append((ppf_value, (x_coord, y_coord)))

# Sort list by highest PPF value (descending order)
ppf_list.sort( key=lambda x: x[0])#,reverse=True)

# Print top regions with highest PPF
for i in range(10):  # Show top 10 high-power locations
    print(f"PPF = {ppf_list[i][0]:.3f} at (X={ppf_list[i][1][0]:.3f}, Y={ppf_list[i][1][1]:.3f})")
#print(ppf_list)

fig, ax = plt.subplots(figsize=(10, 8))  

# Choose an improved colormap
cmap = "RdBu_r"  # Alternatives: "cividis", "plasma", "viridis"

# Plot the heatmap
img = ax.imshow(ppf_grid, cmap=cmap, origin="lower",
                extent=[mesh.lower_left[0], mesh.upper_right[0],
                        mesh.lower_left[1], mesh.upper_right[1]])

# Add colorbar with better positioning
cbar = plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Power Factor ", fontsize=12)

# Compute proper text positions
x_centers = np.linspace(mesh.lower_left[0], mesh.upper_right[0], mesh.dimension[0], endpoint=False) + (mesh.upper_right[0] - mesh.lower_left[0]) / (2 * mesh.dimension[0])
y_centers = np.linspace(mesh.lower_left[1], mesh.upper_right[1], mesh.dimension[1], endpoint=False) + (mesh.upper_right[1] - mesh.lower_left[1]) / (2 * mesh.dimension[1])

# Annotate PPF values inside each cell
for i in range(mesh.dimension[0]):
    for j in range(mesh.dimension[1]):
        if any((guide_tubes_pos == [i, j]).all(axis=1)):  
            continue  # Skip guide tube locations
        ax.text(x_centers[i], y_centers[j], f"{ppf_grid[j, i]:.3f}",  # Swap indices if needed
                ha='center', va='center', fontsize=8, 
                color="black" if ppf_grid[j, i] < 1.0 else "white")  # Adaptive text color

# Overlay guide tube positions
if 'guide_tubes_pos' in locals():
    gt_x = x_centers[guide_tubes_pos[:, 0]]
    gt_y = y_centers[guide_tubes_pos[:, 1]]
    ax.scatter(gt_x, gt_y, color="cyan", s=80, label="Guide Tube")

# Labels and title
ax.set_xlabel("x [cm]", fontsize=12)
ax.set_ylabel("y [cm]", fontsize=12)
ax.set_title("Power distribution map", fontsize=14, fontweight='bold')

plt.show()

In [ ]:
'''
sp = openmc.StatePoint(f"statepoint.20.h5")
print(sp.k_combined.std_dev)
keffs=[]
keff_stddev
keff_stddev =sp.keff.std_dev#= sp.k_combined.std_dev    # Mean and standard deviation of keff
        # Compute relative error
print(keff_stddev)
keffs=sp.keff.nominal_value
print(keff_stddev/keffs)

list1 = [10, 20, 30, 40]  
list2 = [2, 4, 6, 8]  

# Compute element-wise ratio
ratio_list = [a / b for a, b in zip(list1, list2)]

# Print the result
print(ratio_list)

keff_mean = sp.k_combined 
print(keff_mean)
keff_stddev = sp.keff.std_dev    # Mean and standard deviation of keff
        # Compute relative error
print(keff_stddev)
rel_error = keff_stddev / keffs

        # Store values


# Print results
print(rel_error
'''